# Tutorial 3: Making Maps with Cartopy

So far, we've learned how to plot data using x and y axes coordinates, creating line plots, scatter plots, and histograms. Another useful tool for visualizing climate data is plotting the data in space, i.e. making a map. 

In this tutorial, we will learn about the map-making package Cartopy. By the end of this tutorial, you will be able to:
* create a map background of an area of interest
* customize a map for your data
* display data on a map for multiple map projections to best represent your results

### What is Cartopy?

Cartopy is package the focuses on geospatial data. This means it can be used for plotting data on a map, but it can also be used for plotting geographical images such as sections of a Google map. All the info about Cartopy can be found here: https://scitools.org.uk/cartopy/docs/latest/

Like Matplotlib, Cartopy has several different modules. We will be working with two important modules in this tutorial: *crs* and *feature*. crs is the basic map-making module that creates the coordinates that we want. feature allows us to add coastlines, borders, oceans, and other important distinctions to our maps. We'll work through these modules in detail in this tutorial. Then, once we've made the map background, we actually use Matplotlib to plot the data. 

For this tutorial, we will again be using pandas to manipulate our data. We will use Matplotlib for the plotting of data on the Cartopy-created map backgrounds. 

### Data Preparation

**Note:** If you get a <font color=red>ModuleNotFoundError</font> when importing Cartopy modules, then you will need to install this package. Insert a new cell, type "**!pip install cartopy**". Note: in Jupyter notebooks, the exclamation mark (!) at the beginning of a line in a code cell indicates that the following command is a shell command

In [ ]:
!pip install cartopy

In [ ]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cartopy import crs, feature # we only want the crs and feature modules from cartopy

In [ ]:
# we'll be using our weather station data again
csvfile_pk = 'LAHORE_PK.csv' 
csvfile_pa = 'PHILADELPHIA_US.csv'

# read the data
df_pk = pd.read_csv(csvfile_pk, sep=',')
df_pa = pd.read_csv(csvfile_pa, sep=',')
df_pk

### Making a Map

There are a few steps to making a map. 
1. Identify the coordinates of your area of interest
2. Make a grid on your coordinates of interest
3. Select a map projection
4. Create a pyplot figure using your grid and map projection
5. Add the relevant features to your figure
6. Plot the data

#### 1. Identify the coordinates
In some cases, your data will have a pre-made grid of the latitude and longitude coordinates for you. This makes it really easy. In other cases, you might need to decide on the coordinates yourself. Our data does have latitude and longitude coordinates, but it does not tell us the size of the grid we want. We can use our latitude and longitude data to make an appropriately sized grid.

In [ ]:
latmin_pk = df_pk['LATITUDE'].min()
latmax_pk = df_pk['LATITUDE'].max()
lonmin_pk = df_pk['LONGITUDE'].min()
lonmax_pk = df_pk['LONGITUDE'].max()
print('Lat Range: %.4f - %.4f' % (latmin_pk,latmax_pk))
print('Lon Range: %.4f - %.4f' % (lonmin_pk,lonmax_pk))

In [ ]:
latmin_pa = df_pa['LATITUDE'].min()
latmax_pa = df_pa['LATITUDE'].max()
lonmin_pa = df_pa['LONGITUDE'].min()
lonmax_pa = df_pa['LONGITUDE'].max()
print('Lat Range: %.4f - %.4f' % (latmin_pa,latmax_pa))
print('Lon Range: %.4f - %.4f' % (lonmin_pa,lonmax_pa))

Since Lahore only has one station, we’ll use Philadelphia, which has multiple stations, for this exercise to generate the geographic extent. Now we know the max and min values of our coordinates. This will help us to make a box around our data. It is best to round up/down to give a little bit of buffer space around the data.

In [ ]:
# buffered lat/lon values
latmin = 39.8
latmax = 40.1
lonmin = -75.3
lonmax = -74.95

#### 2. Make a grid

This step will vary depending on the type of data you are plotting. We will be making a simple dot map, so we basically need to set a max and min range for our map's box size. If we were using gridded data that covered the whole area, like satellite data or a model output, we would need to make a meshgrid that had a lat and lon coordinate pair for each grid cell. We will go through that process in a later tutorial.

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

#### 3. Select a map projection

We also need to decide on a map projection. A map projection tells you how the map will be displayed. Check out this website for examples: https://scitools.org.uk/cartopy/docs/latest/crs/projections.html

In [ ]:
# let's set our projection as PlateCarree
proj = crs.PlateCarree()

#### 4. Create a figure

Now that we have the basics ready, we can put them together in a pyplot figure to set up our map background. 

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,8))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
# ax.set_extent([lonmin,lonmax,latmin,latmax],crs.PlateCarree())
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
plt.show()

Woot! A box! How can we tell if this box is set up correctly? We can add feature details like rivers, state borders, etc. to determine if it's in the right place.

#### 5. Add relevant features

**Note:** Depending on which version of Cartopy you have installed, you may get some download warnings or errors with the features. 
* If you get errors and the map does not show, run a "!pip install --update-all cartopy" command in a new cell.
* If you are getting Download Warnings but the figure does appear, then don't worry. If you rerun the cell, you will see the Download Warning has disappeared. If you add new features, you will get a new warning, but it'll go away the next time you use that feature. When Cartopy was initially installed, it didn't download all the different map features with it. So it has to download them the first time you want to use those features in a plot.

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(12,4))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# add features: 
ax.add_feature(feature.STATES)
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')

plt.show()

Success! That is definitely a box around Philadephia! If you want to change the area of the map, go back to step 1 and select new lat and lon values. 

Ok, now to plot the data!

#### 6. Plot the data

The daily ozone data has lat and lon coordinates. But we have data for every day in the month of July, which is too much to plot all at once. Instead, let's pick just one day and plot that data first. 

In [ ]:
# select monitor data for a specific day
sel_day = df_pa[df_pa['DATE'] == '2023-07-16']
sel_day

In [ ]:
# get the lat, lon, and y-data
lats = sel_day['LATITUDE']
lons = sel_day['LONGITUDE']
TMAX = sel_day['TMAX']
PRCP = sel_day['PRCP']

In [ ]:
# create a pyplot figure for TMAX
fig = plt.figure(figsize=(8,8))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# manipulate gridliner object
gl.xlabels_top = False
gl.ylabels_right = False

# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')

# plot the data -- note, we use plt.scatter to add data on top of the map!
plt.scatter(lons, lats, s=100, c=TMAX, cmap='viridis', edgecolors='black', linewidth=1.5) # Check out more Colormaps options here: https://matplotlib.org/stable/users/explain/colors/colormaps.html
plt.colorbar(label='TMAX ($^\circ$F)', shrink=0.69)
plt.show()

Awesome work, everyone! 🎉

We’ve created a map, plotted our data, and now we can start analyzing what it shows.

From this map, it’s clear that the highest daily maximum temperature (TMAX) was recorded near the Philadelphia Franklin Institute (96°F), followed by Northeast Philadelphia Airport and Philadelphia International Airport.

You might also notice that three days have missing data, shown as empty circles on the map.

Now, let’s repeat this process with the precipitation (PRCP) data to see what patterns we can find there.

In [ ]:
# create a pyplot figure for PRCP
fig = plt.figure(figsize=(8,8))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# manipulate gridliner object
gl.xlabels_top = False
gl.ylabels_right = False

# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')

# plot the data -- note, we use plt.scatter to add data on top of the map!
plt.scatter(lons, lats, s=100, c=PRCP, cmap='viridis', edgecolors='black', linewidth=1.5) # Check out more Colormaps options here: https://matplotlib.org/stable/users/explain/colors/colormaps.html
plt.colorbar(label='PRCP (inch)', shrink=0.69)
plt.show()

Now we can see that the occurrence of precipitation at Northeast Philadelphia Airport and Philadelphia International Airport might be the reason why their TMAX values remained relatively lower.

### The importance of projections

Now that we've learned how to make a map, let's dig a little deeper into some of the details of Cartopy. Specifically, what is going on with the map projections? What do different projections look like? How does this affect the data?

To understand the importance of map projections, we need to look at a much larger scale than just the city level. Let’s switch our focus to data covering the entire United States – in fact, the dataset we’re about to use covers the whole globe!

This dataset is quite large and was downloaded from the https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/. Be aware that downloading it may take some time due to its size.

In [ ]:
# specify file path
csvfile = 'ghcnd_2023.csv' 
csvfile_stations =  'ghcnd-stations.csv'

Since the data file does not include a header, we first need to manually specify the column headers before working with the data.

In [ ]:
######################## https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/readme-by_year.txt
### ID = 11 character station identification code
### YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
### ELEMENT = 4 character indicator of element type 
### DATA VALUE = 5 character data value for ELEMENT 
### M-FLAG = 1 character Measurement Flag 
### Q-FLAG = 1 character Quality Flag 
### S-FLAG = 1 character Source Flag 
### OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)
################################################################################################  
colnames=['STATION', 'DATE', 'ELEMENT', 'VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
df = pd.read_csv(csvfile, names=colnames, header=None)
df

This dataset has a different data structure compared to what we used in our previous tutorial. To explore and understand what it contains, we can use the .unique() function to check the unique values within the DATE and ELEMENT columns.

In [ ]:
df['DATE'].unique()

In [ ]:
df['ELEMENT'].unique()

Again, we will focus on ‘TMAX’ (daily maximum temperature) and ‘PRCP’ (precipitation) in our analysis. Let’s also narrow down our time period to July.

In [ ]:
#### ------------------------------
#### Variable      Type
#### ------------------------------
#### ID            Character
#### YEAR          Integer
#### MONTH         Integer
#### ELEMENT       Character
#### VALUE         Integer
#### MFLAG         Character
#### QFLAG         Character
#### SFLAG         Character

# Select TMAX
df_TMAX = df[df['ELEMENT']=='TMAX']
df_TMAX_July = df_TMAX[(df_TMAX['DATE'] >= 20230701) & (df_TMAX['DATE'] <= 20230731)]

# Select PRCP
df_PRCP = df[df['ELEMENT']=='PRCP']
df_PRCP_July = df_PRCP[(df_PRCP['DATE'] >= 20230701) & (df_PRCP['DATE'] <= 20230731)]
df_PRCP_July

Note: This dataset does not include latitude and longitude information. Therefore, our next step will be to retrieve this information from the GHCND stations file, which contains the geographic coordinates for each station.

In [ ]:
# Read stations
colnames_stations=['STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME']
df_stations = pd.read_csv(csvfile_stations, names=colnames_stations, header=None, index_col=False)
df_stations

Now, let's use the **.merge()** function to combine our dataset with the station coordinate information, using "STATION" as the common key. This will allow us to add the latitude and longitude data to our main dataset.

In [ ]:
df_TMAX_July_coord = pd.merge(df_TMAX_July, df_stations, on="STATION", how="left")
df_PRCP_July_coord = pd.merge(df_PRCP_July, df_stations, on="STATION", how="left")
df_PRCP_July_coord

Let’s check if any coordinate information is missing from our merged dataset.

In [ ]:
print(df_TMAX_July_coord[df_TMAX_July_coord['LATITUDE'].isnull()==True]['STATION'].unique())
print(df_TMAX_July_coord[df_TMAX_July_coord['LONGITUDE'].isnull()==True]['STATION'].unique())

Great job so far! Next, let’s filter our data to keep only high-quality observations.
We can refer to Table 3 (Source Flag/Attribute): https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/GHCND_documentation.pdf to understand the quality codes.

In [ ]:
df_TMAX_July_coord_qflag = df_TMAX_July_coord[df_TMAX_July_coord['Q-FLAG'].isnull()==True]
df_PRCP_July_coord_qflag = df_PRCP_July_coord[df_PRCP_July_coord['Q-FLAG'].isnull()==True]

Now, let’s check the geographical extent of our data. This will help us understand the spatial coverage and define appropriate boundaries for mapping and analysis.

In [ ]:
# what are the min and max coordinates of monitors in our dataset?
latmin = df_TMAX_July_coord_qflag['LATITUDE'].min()
latmax = df_TMAX_July_coord_qflag['LATITUDE'].max()
lonmin = df_TMAX_July_coord_qflag['LONGITUDE'].min()
lonmax = df_TMAX_July_coord_qflag['LONGITUDE'].max()
print('Lat Range: %.4f - %.4f' % (latmin,latmax))
print('Lon Range: %.4f - %.4f' % (lonmin,lonmax))

This dataset covers almost the entire globe, so let's define the global extent

In [ ]:
# define NA bounding box 
latmin = -90  
latmax = 90   
lonmin = -180 
lonmax = 180  

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

Now, let’s filter the data to include only those days with “scorching temperatures,” which we define as temperatures greater than 45°C.

Note: The temperature values in this dataset are recorded in tenths of degrees Celsius, so a temperature of 45°C is represented as 450. Therefore, our threshold for filtering TMAX will be greater than 450.

In [ ]:
df_TMAX_July_coord_qflag_thres = df_TMAX_July_coord_qflag[df_TMAX_July_coord_qflag['VALUE'] > 450]
df_TMAX_July_coord_qflag_thres_sorted = df_TMAX_July_coord_qflag_thres.sort_values(by='VALUE', ascending=True)
df_TMAX_July_coord_qflag_thres_sorted

In [ ]:
# PlateCarree
proj = crs.PlateCarree()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
cmap_TMAX = 'hot_r'

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# manipulate gridliner object
gl.xlabels_top = False
gl.ylabels_right = False

# add features
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')
ax.add_feature(feature.COASTLINE)

# plot the data -- note, we use plt.scatter to add data on top of the map!
plt.scatter(df_TMAX_July_coord_qflag_thres_sorted['LONGITUDE'], df_TMAX_July_coord_qflag_thres_sorted['LATITUDE'], 
           c=df_TMAX_July_coord_qflag_thres_sorted['VALUE']/10.0, s=30, cmap=cmap_TMAX, edgecolors='black')
plt.colorbar(label='TMAX ($^\circ$F)', shrink=0.33)

plt.show()

Ta da! We have now transformed our flat data so that it can be plotted on a curved map. Basically any time you plot data onto a large map using a non-flat projection, you will need to add a transformation. 

Let’s apply a similar filter for precipitation data. This time, we want to select only the days with heavy precipitation, defined as more than 50 mm.

Note: If the precipitation values are recorded in tenths of millimeters, then 50 mm would be represented as 500 in the dataset.

So, we will filter PRCP for values greater than 500.

In [ ]:
df_PRCP_July_coord_qflag_thres = df_PRCP_July_coord_qflag[df_PRCP_July_coord_qflag['VALUE'] > 500]
df_PRCP_July_coord_qflag_thres_sorted = df_PRCP_July_coord_qflag_thres.sort_values(by='VALUE', ascending=True)
df_PRCP_July_coord_qflag_thres_sorted

In [ ]:
# PlateCarree
proj = crs.PlateCarree()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
cmap_PRCP = 'cool'

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# manipulate gridliner object
gl.xlabels_top = False
gl.ylabels_right = False

# add features
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')
ax.add_feature(feature.COASTLINE)

# plot the data -- note, we use plt.scatter to add data on top of the map!
plt.scatter(df_PRCP_July_coord_qflag_thres_sorted['LONGITUDE'], df_PRCP_July_coord_qflag_thres_sorted['LATITUDE'], 
           c=df_PRCP_July_coord_qflag_thres_sorted['VALUE']/10.0, s=30, cmap=cmap_PRCP, edgecolors='black',vmin=50, vmax=300,)
plt.colorbar(label='PRCP (mm)', shrink=0.33)

plt.show()

### Bonus

What do you notice about the shape of this map? Does the map seem to be the right shape? Or do things look a bit flattened? That's what happens with PlateCarree, Let's use United States as an example to try some other projections. 

In [ ]:
# buffered lat and lon values
latmin = 15
latmax = 65
lonmin = -60
lonmax = -150

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

In [ ]:
# Albers Equal Area projection
proj = crs.AlbersEqualArea()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# add features
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')
ax.add_feature(feature.COASTLINE)
plt.show()

That's pretty funky! The Albers Equal Area projection is a conical projection, meaning it pretends the globe is actually a cone. This is how it accounts for curvature on our flat computer screens.

In [ ]:
# Lambert Conformal. This projection is useful if you ever use CMAQ or WRF data
proj = crs.LambertConformal()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# add features
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')
ax.add_feature(feature.COASTLINE)
plt.show()

Again, there is curvature in the map. But this time the US is centered a bit better. 

In [ ]:
# Robinson projection
proj = crs.Robinson()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))

# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)

# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')

# add features
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN, facecolor='lightblue')
ax.add_feature(feature.COASTLINE)
plt.show()

### Exercises

**1.** Now it's your turn! Pick any location you’re interested in and follow the steps above to plot the data on a local map. You can set your map to be as zoomed-in (small area) or zoomed-out (large area) as you like. Make sure to include all the monitoring stations in your selected area.

**2.** Choose any other date that interests you and repeat the steps above to plot the data on a global map. Observe how the patterns change over time and think about:
- What differences do you notice compared to other dates?
- Are there regions that consistently show extreme temperatures or precipitation?

**3.** Try to make a map of the United States (or other selected countries) using a new projection from https://scitools.org.uk/cartopy/docs/latest/crs/projections.html. 